# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Hemochromatosis'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [24]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [25]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [26]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [27]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [28]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [29]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [30]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [31]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [32]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [33]:
gender_col = 'gender'

In [34]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [35]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

15 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('SNORD104', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
9 input query terms found dup hits:	[('SUGT1P1', 2), ('SNORD127', 2), ('SNORA62', 4), ('IFITM4P', 8), ('HLA-DRB6', 2), ('FUNDC2P2', 2), 
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
20 input query terms found dup hits:	[('PIP5K1P1', 2), ('SNORA40', 2), ('SNORD116-24', 2), ('SNORD116-25', 2), ('SNORD116-23', 2), ('SNOR
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
18 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('SNORD116-22', 2), (
158 input query terms found no hit:	['

In [36]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [37]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [38]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [39]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [40]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "Hemochromatosis"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE50579', 'GSE20713', 'GSE159676', 'GSE129322']

Repeat the below steps for all the accession numbers

In [23]:
cohort = accession_num = "GSE50579"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Hemochromatosis/GSE50579/GSE50579_family.soft.gz',
 '/home/yxcheng/GEO/Hemochromatosis/GSE50579/GSE50579_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [24]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Expression profiling of HCC"
!Series_summary	"61 human HCCs were analyzed for genome-wide gene expression. Samples were collected at two sites in Germany, Heidelberg (HD) and Hannover (N). The Heidelberg Collection include 40 independent HCC: 19 liver resections and 17 explant liver specimen (4 not determined); median age at surgery was 57 years (range, 16-78), and the male/female ratio was 3:1. All diagnoses were confirmed by histological reevaluation, and use of the samples was approved by the local ethics committee.  From 3 patients, two HCC nodules were included that previously showed different aCGH, indicating independent tumor development (tagged by _2). The underlying etiologies were HBV (n=8), HCV (n=9), alcohol (n=14), cryptogenic (n=11), genetic hemochromatosis (n=3), and Alpha-1 antitrypsin deficiency (n=2). The Hannover Collection include 21 HCC (HCC-Grad: G1-G3). Technical replicates were tagged with _B or _C"
!Series_overall_design	"differential genes expre

In [25]:
clinical_data.head()

,!Sample_geo_accession,GSM1223814,GSM1223815,GSM1223816,GSM1223817,GSM1223818,GSM1223819,GSM1223820,GSM1223821,GSM1223822,...,GSM1223884,GSM1223885,GSM1223886,GSM1223887,GSM1223888,GSM1223889,GSM1223890,GSM1223891,GSM1223892,GSM1223893
0,!Sample_characteristics_ch1,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,...,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.,tumor stage: HCC-Grad:G2,tumor stage: HCC-Grad:G2/3,tumor stage: HCC-Grad:G2,tumor stage: n.d.,tumor stage: n.d.,tumor stage: n.d.
1,!Sample_characteristics_ch1,etiology: alcohol abuse,etiology: alcohol abuse,etiology: alcohol abuse,etiology: cryptogenic,etiology: cryptogenic,etiology: cryptogenic,etiology: HCV,etiology: HBV,etiology: cryptogenic,...,etiology: n.d.,etiology: n.d.,etiology: alcohol abuse,etiology: Alpha-1 antitrypsin deficiency,etiology: n.d.,etiology: n.d.,etiology: n.d.,etiology: n.d.,etiology: n.d.,etiology: n.d.
2,!Sample_characteristics_ch1,sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),...,sample type: Normal liver tissue control,sample type: Normal liver tissue control,sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: hepatocellular carcinoma (HCC),sample type: Normal liver tissue control,sample type: Normal liver tissue control,sample type: Normal liver tissue control
3,!Sample_characteristics_ch1,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: female,gender: female,...,gender: male,gender: female,gender: male,gender: male,gender: n.d.,gender: n.d.,gender: n.d.,gender: n.d.,gender: female,gender: n.d.
4,!Sample_characteristics_ch1,material: Tx,material: Tx,material: Tx,material: n.d.,material: Rx,material: Rx,material: Tx,material: Tx,material: Rx,...,material: Rx,material: Rx,material: n.d.,material: Tx,material: n.d.,material: n.d.,material: n.d.,material: n.d.,material: Rx,material: n.d.


In [8]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['gender: F', 'gender: M', 'gender: NA'],
 1: ['age: 74',
  'age: 18',
  'age: 35',
  'age: 63',
  'age: 30',
  'age: 59',
  'age: 39',
  'age: 64',
  'age: 67',
  'age: 56',
  'age: 51',
  'age: 65',
  'age: 69',
  'age: 40',
  'age: 60',
  'age: 43',
  'age: 55',
  'age: 47',
  'age: 68',
  'age: 53',
  'age: 62',
  'age: 57',
  'age: 49',
  'age: 73',
  'age: 78',
  'age: 75',
  'age: 66',
  'age: 79',
  'age: 52',
  'age: 77'],
 2: ['diagnosis: HCC'],
 3: ['type: RESECTION',
  'type: TRANSPLANT',
  'type: RADIOFREQ',
  'type: ADVANCED'],
 4: ['without.etiology: no', 'without.etiology: yes', 'without.etiology: NA'],
 5: ['alcohol: no', 'alcohol: yes', 'alcohol: NA'],
 6: ['hepatitis.b: yes', 'hepatitis.b: no', 'hepatitis.b: NA'],
 7: ['hepatitis.c: no', 'hepatitis.c: yes', 'hepatitis.c: NA'],
 8: ['hemochromatosis: no', 'hemochromatosis: yes', 'hemochromatosis: NA'],
 9: ['metabolic.syndrome: no',
  'metabolic.syndrome: NA',
  'metabolic.syndrome: yes'],
 10: ['metavir fibrosis:

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [31]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Hemochromatosis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Hemochromatosis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [9]:
is_gene_availabe = True
trait_row_id = 2
age_row_id = 1
gender_row_id = 0

trait_type = 'binary'

In [10]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [11]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'diagnosis: HCC':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age:'):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: F':
        return 0
    elif gender_string == 'gender: M':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'sample type: hepatocellular carcinoma (HCC)'
example_age = 'age (yrs): 47'
example_gender = 'gender: male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [15]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, convert_trait, age_row=age_row_id,
                                                      convert_age=convert_age, gender_row=gender_row_id,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM3704522,GSM3704523,GSM3704524,GSM3704525,GSM3704526,GSM3704527,GSM3704528,GSM3704529,GSM3704530,GSM3704531,...,GSM3705199,GSM3705200,GSM3705201,GSM3705202,GSM3705203,GSM3705204,GSM3705205,GSM3705206,GSM3705207,GSM3705208
Hemochromatosis,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Age,74,18,35,63,30,59,35,39,39,63,...,57,54,55,55,55,55,55,55,69,77
Gender,0,0,1,1,1,0,0,1,0,1,...,1,1,1,1,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [16]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM3704522,GSM3704523,GSM3704524,GSM3704525,GSM3704526,GSM3704527,GSM3704528,GSM3704529,GSM3704530,GSM3704531,...,GSM3705199,GSM3705200,GSM3705201,GSM3705202,GSM3705203,GSM3705204,GSM3705205,GSM3705206,GSM3705207,GSM3705208
ID,,,,,,,,,,,,,,,,,,,,,


In [15]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['cg00000029',
 'cg00000108',
 'cg00000109',
 'cg00000165',
 'cg00000236',
 'cg00000289',
 'cg00000292',
 'cg00000321',
 'cg00000363',
 'cg00000622',
 'cg00000658',
 'cg00000714',
 'cg00000721',
 'cg00000734',
 'cg00000769',
 'cg00000807',
 'cg00000884',
 'cg00000905',
 'cg00000924',
 'cg00000948']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [16]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622', 'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769', 'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948']\n"

If not required, jump directly to the gene normalization step

In [17]:
requires_gene_mapping = True

In [18]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, '31717405', nan, nan, nan], 'AlleleB_ProbeSeq': [nan, 'ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAACCCCG', nan, nan, nan], 'Infinium_Design_Type': ['II', 'I', 'II', 'II', 'II'], 'Next_Base': [nan, 'A', nan, nan, nan], 'Color_Channel': [nan, 'Red', nan, nan, nan], 'Forward_Sequence': ['AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCTGCTCAGCTGGAAAC[CG]AATTTGAGATAAATTCAAGGGTCTATGTGGACAAGACTGCTAGTGTCTCTCTCTGGATTG', 'TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTG

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [19]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, '31717405', nan, nan, nan], 'AlleleB_P

In [20]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [21]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [22]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [23]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 15 samples.


In [24]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Epilepsy', the least common label is '1.0' with 6 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 24.5
  50% (Median): 39.0
  75%: 60.5
Min: 13.0
Max: 77.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Haoyang/Epilepsy/cohort_info.json


In [26]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [41]:
from utils import *

In [42]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'Xena'

In [43]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,1100,


In [44]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [45]:
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
1,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
2,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
3,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
4,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [46]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [47]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [48]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [49]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [50]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 59.27% ± 1.43%


In [52]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 90.09% ± 2.36%


In [53]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [55]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 10 genes with non-zero coefficients associated with the trait 'Lung-Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
   Variable  Coefficient  Absolute Coefficient
    SNORA81     0.605617              0.605617
    SNORD89    -0.561992              0.561992
     OR4F17     0.468452              0.468452
SNORD115-26    -0.289358              0.289358
     OR2L1P    -0.278115              0.278115
    TBC1D3C     0.248994              0.248994
     DMRTA2    -0.194621              0.194621
     IFIT1B    -0.188779              0.188779
    PPIAL4E     0.150659              0.150659
     TTTY16    -0.082366              0.082366


/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [56]:

interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 14 significant genes associated with the trait 'Lung-Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
   MEX3A     0.061062 2.147189e-07           0.003723
 SLC19A1    -0.038907 6.271788e-07           0.005437
  CRABP2     0.036440 1.633672e-06           0.009442
   HMGB3     0.042606 8.393633e-06           0.020791
 DEFB118     0.028296 7.138943e-06           0.020791
     IL3    -0.016966 5.077872e-06           0.020791
  FAM83A     0.043838 8.074414e-06           0.020791
    RCC1     0.041270 1.786386e-05           0.028158
    NUF2     0.068410 1.754965e-05           0.028158
    SPP1     0.032014 1.782137e-05           0.028158
    ETV4     0.037361 1.688133e-05           0.028158
   EPB42    -0.020447 2.157070e-05           0.031168
   CENPM    -0.051850 3.061145e-05           0.040829
 PLEKHN1     0.044331 3.372218

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
